In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trainandtest/train.csv
/kaggle/input/trainandtest/test.csv


In [2]:
import sklearn
import numpy as np
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from prettytable import PrettyTable
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.get_lock().locks = []
# !pip install sweetviz
# import sweetviz as sv
import concurrent.futures
from copy import deepcopy       
from functools import partial
from itertools import combinations
import random
from random import randint, uniform
import gc
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from itertools import combinations
from sklearn.impute import SimpleImputer
import xgboost as xg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
from sklearn.cluster import KMeans
!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer
!pip install gap-stat
from gap_statistic.optimalK import OptimalK
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind
from scipy.stats import boxcox
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.base import BaseEstimator, TransformerMixin
!pip install optuna
import optuna
import xgboost as xgb
!pip install catboost
!pip install lightgbm --install-option=--gpu --install-option="--boost-root=C:/local/boost_1_69_0" --install-option="--boost-librarydir=C:/local/boost_1_69_0/lib64-msvc-14.1"
import lightgbm as lgb
!pip install category_encoders
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier,ExtraTreesClassifier, AdaBoostClassifier
!pip install -U imbalanced-learn
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from catboost import Pool
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option('display.max_columns',None)


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --install-option


/opt/conda/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [3]:
import pandas as pd
train = pd.read_csv('/kaggle/input/trainandtest/train.csv')
test = pd.read_csv('/kaggle/input/trainandtest/test.csv')

In [4]:
train.drop(columns=["id"],inplace=True)
test.drop(columns=["id"],inplace=True)
train['Target'] = train['Target'].map({'Graduate': 1, 'Dropout': 0, 'Enrolled': 2})
train.head()


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,1,1,9238,1,1,126.0,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.500000,0,0,6,7,6,12.428571,0,11.1,0.6,2.02,1
1,1,17,1,9238,1,1,125.0,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.600000,0,0,6,9,0,0.000000,0,11.1,0.6,2.02,0
2,1,17,2,9254,1,1,137.0,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92,0
3,1,1,3,9500,1,1,131.0,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.591250,0,0,8,11,7,12.820000,0,11.1,0.6,2.02,2
4,1,1,2,9500,1,1,132.0,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.933333,0,0,7,12,6,12.933333,0,7.6,2.6,0.32,1


In [5]:
threshold = 10
train['grades_interaction'] = train['Curricular units 1st sem (grade)'] * train['Curricular units 2nd sem (grade)']
train['grade_ratio'] =train['Curricular units 1st sem (grade)'] / (train['Curricular units 2nd sem (grade)'] + 1)
train['total_grades'] = train['Curricular units 1st sem (grade)'] + train['Curricular units 2nd sem (grade)']
train['total_approved_units'] =train['Curricular units 1st sem (approved)'] + train['Curricular units 2nd sem (approved)']
train['performance_index'] = (train['Curricular units 1st sem (grade)'] + train['Curricular units 2nd sem (grade)']) / (train['Curricular units 1st sem (evaluations)'] +train['Curricular units 2nd sem (evaluations)'] + 1)
train['parent_qualification'] = (train['Father\'s qualification'] +train['Mother\'s qualification']) / 2
train['qualification_diff'] = abs(train['Father\'s qualification'] - train['Mother\'s qualification'])
train['high_father_qualification'] = train['Father\'s qualification'].apply(lambda x: 1 if x > threshold else 0)
train['high_mother_qualification'] = train['Mother\'s qualification'].apply(lambda x: 1 if x > threshold else 0)
train['both_high_qualification'] = train.apply(lambda row: 1 if row['Father\'s qualification'] > threshold and row['Mother\'s qualification'] > threshold else 0, axis=1)

train['unemployment_gdp_interaction'] = train['Unemployment rate'] * train['GDP']
train['normalized_gdp'] = (train['GDP'] - train['GDP'].min()) / (train['GDP'].max() - train['GDP'].min())
train['normalized_unemployment'] = (train['Unemployment rate'] - train['Unemployment rate'].min()) / (train['Unemployment rate'].max() - train['Unemployment rate'].min())
train['normalized_inflation'] = (train['Inflation rate'] - train['Inflation rate'].min()) / (train['Inflation rate'].max() - train['Inflation rate'].min())

train['stability_index'] = 1 / (1 + train['Unemployment rate'] + train['Inflation rate'])

# Applying feature engineering to the test dataset
test['grades_interaction'] = test['Curricular units 1st sem (grade)'] * test['Curricular units 2nd sem (grade)']
test['grade_ratio'] = test['Curricular units 1st sem (grade)'] / (test['Curricular units 2nd sem (grade)'] + 1)
test['total_grades'] = test['Curricular units 1st sem (grade)'] + test['Curricular units 2nd sem (grade)']
test['total_approved_units'] = test['Curricular units 1st sem (approved)'] + test['Curricular units 2nd sem (approved)']
test['performance_index'] = (test['Curricular units 1st sem (grade)'] + test['Curricular units 2nd sem (grade)']) / (test['Curricular units 1st sem (evaluations)'] + test['Curricular units 2nd sem (evaluations)'] + 1)
test['parent_qualification'] = (test['Father\'s qualification'] + test['Mother\'s qualification']) / 2
test['qualification_diff'] = abs(test['Father\'s qualification'] - test['Mother\'s qualification'])
test['high_father_qualification'] = test['Father\'s qualification'].apply(lambda x: 1 if x > threshold else 0)
test['high_mother_qualification'] = test['Mother\'s qualification'].apply(lambda x: 1 if x > threshold else 0)
test['both_high_qualification'] = test.apply(lambda row: 1 if row['Father\'s qualification'] > threshold and row['Mother\'s qualification'] > threshold else 0, axis=1)

test['unemployment_gdp_interaction'] = test['Unemployment rate'] * test['GDP']
test['normalized_gdp'] = (test['GDP'] - test['GDP'].min()) / (test['GDP'].max() - test['GDP'].min())
test['normalized_unemployment'] = (test['Unemployment rate'] - test['Unemployment rate'].min()) / (test['Unemployment rate'].max() - test['Unemployment rate'].min())
test['normalized_inflation'] = (test['Inflation rate'] - test['Inflation rate'].min()) / (test['Inflation rate'].max() - test['Inflation rate'].min())

test['stability_index'] = 1 / (1 + test['Unemployment rate'] + test['Inflation rate'])


In [6]:
def better_features(train, test, target_col, cols, best_score):
    new_cols = []
    skf = KFold(n_splits=5, shuffle=True, random_state=42)  # Stratified k-fold object

    for i in tqdm(range(len(cols)), desc='Generating Columns'):
        col1 = cols[i]
        temp_df = pd.DataFrame()  # Temporary dataframe to store the generated columns
        temp_df_test = pd.DataFrame()  # Temporary dataframe for test data

        for j in range(i + 1, len(cols)):
            col2 = cols[j]
            # Multiply
            temp_df[col1 + '*' + col2] = train[col1] * train[col2]
            temp_df_test[col1 + '*' + col2] = test[col1] * test[col2]

            # Divide (col1 / col2)
            temp_df[col1 + '/' + col2] = train[col1] / (train[col2] + 1e-5)
            temp_df_test[col1 + '/' + col2] = test[col1] / (test[col2] + 1e-5)

            # Divide (col2 / col1)
            temp_df[col2 + '/' + col1] = train[col2] / (train[col1] + 1e-5)
            temp_df_test[col2 + '/' + col1] = test[col2] / (test[col1] + 1e-5)

            # Subtract
            temp_df[col1 + '-' + col2] = train[col1] - train[col2]
            temp_df_test[col1 + '-' + col2] = test[col1] - test[col2]

            # Add
            temp_df[col1 + '+' + col2] = train[col1] + train[col2]
            temp_df_test[col1 + '+' + col2] = test[col1] + test[col2]

        SCORES = []
        for column in temp_df.columns:
            scores = []
            for train_index, val_index in skf.split(train, train[target_col]):
                X_train, X_val = temp_df[column].iloc[train_index].values.reshape(-1, 1), temp_df[column].iloc[val_index].values.reshape(-1, 1)
                y_train, y_val = train[target_col].astype(int).iloc[train_index], train[target_col].astype(int).iloc[val_index]
                model = LogisticRegression(multi_class='ovr', solver='liblinear')
                model.fit(X_train, y_train)
                y_pred = model.predict_proba(X_val)
                score = roc_auc_score(y_val, y_pred, multi_class='ovr')
                scores.append(score)
            mean_score = np.mean(scores)
            SCORES.append((column, mean_score))

        if SCORES:
            best_col, best_auc = sorted(SCORES, key=lambda x: x[1], reverse=True)[0]
            corr_with_other_cols = train.drop([target_col] + new_cols, axis=1).corrwith(temp_df[best_col])
            if (corr_with_other_cols.abs().max() < 0.9 or best_auc > best_score) and corr_with_other_cols.abs().max() != 1:
                train[best_col] = temp_df[best_col]
                test[best_col] = temp_df_test[best_col]
                new_cols.append(best_col)
                print(f"Added column '{best_col}' with ROC AUC Score: {best_auc:.4f} & Correlation {corr_with_other_cols.abs().max():.4f}")

    return train, test, new_cols

In [ ]:
global overall_best_score
global overall_best_col
overall_best_score=0
overall_best_col='none'
selected_features=[f for f in train.columns if train[f].nunique()>2]
selected_features.remove("Target")
train, test,new_cols=better_features(train, test,"Target",  selected_features, overall_best_score)

Generating Columns:   2%|▎         | 1/40 [01:58<1:17:17, 118.91s/it]

Added column 'Marital status-total_approved_units' with ROC AUC Score: 0.8802 & Correlation 0.9967


Generating Columns:   5%|▌         | 2/40 [04:04<1:17:46, 122.80s/it]

Added column 'Application mode-grades_interaction' with ROC AUC Score: 0.8562 & Correlation 0.9803


Generating Columns:   8%|▊         | 3/40 [06:54<1:29:08, 144.55s/it]

...

Added column 'Curricular units 1st sem (credited)-total_approved_units' with ROC AUC Score: 0.8854 & Correlation 0.9759


Generating Columns:  38%|███▊      | 15/40 [27:48<40:04, 96.18s/it]

Added column 'Curricular units 1st sem (enrolled)-total_approved_units' with ROC AUC Score: 0.8882 & Correlation 0.9552


Generating Columns:  40%|████      | 16/40 [29:14<37:17, 93.21s/it]

Added column 'Curricular units 1st sem (evaluations)-total_grades' with ROC AUC Score: 0.8665 & Correlation 0.9450


In [ ]:
print(new_cols)

In [ ]:
kek =['Marital status-Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (approved)/Application mode', 'Application order+Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (approved)/Course', 'Curricular units 2nd sem (approved)/Previous qualification', 'Previous qualification (grade)*Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (approved)/Nacionality', "Curricular units 2nd sem (approved)/Mother's qualification", "Father's qualification*Curricular units 2nd sem (approved)", "Mother's occupation*Curricular units 2nd sem (approved)", "Father's occupation*Curricular units 2nd sem (approved)", 'Admission grade*Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (approved)/Age at enrollment', 'Curricular units 1st sem (credited)-Curricular units 2nd sem (approved)', 'Curricular units 1st sem (enrolled)*Curricular units 2nd sem (approved)', 'Curricular units 1st sem (evaluations)-Curricular units 2nd sem (grade)', 'Curricular units 1st sem (approved)+Curricular units 2nd sem (approved)', 'Curricular units 1st sem (grade)+Curricular units 2nd sem (approved)', 'Curricular units 1st sem (without evaluations)-Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (credited)-Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (enrolled)*Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (evaluations)-Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (approved)+Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (grade)-Curricular units 2nd sem (without evaluations)', 'Curricular units 2nd sem (without evaluations)-GDP', 'Unemployment rate*GDP', 'Inflation rate-GDP']
print(len(kek))

In [ ]:
def apply_arithmetic_operations(train_df, test_df, expressions_list):
    '''
    We pass the selected arithmetic combinations
    '''
    for expression in expressions_list:
        if expression not in train_df.columns:
            # Split the expression based on operators (+, -, *, /)
            parts = expression.split('+') if '+' in expression else \
                    expression.split('-') if '-' in expression else \
                    expression.split('*') if '*' in expression else \
                    expression.split('/')

            # Get the DataFrame column names involved in the operation
            cols = [col for col in parts]

            # Perform the corresponding arithmetic operation based on the operator in the expression
            if cols[0] in train_df.columns and cols[1] in train_df.columns:
                if '+' in expression:
                    train_df[expression] = train_df[cols[0]] + train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] + test_df[cols[1]]
                elif '-' in expression:
                    train_df[expression] = train_df[cols[0]] - train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] - test_df[cols[1]]
                elif '*' in expression:
                    train_df[expression] = train_df[cols[0]] * train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] * test_df[cols[1]]
                elif '/' in expression:
                    train_df[expression] = train_df[cols[0]] / (train_df[cols[1]]+1e-5)
                    test_df[expression] = test_df[cols[0]] /( test_df[cols[1]]+1e-5)
    
    return train_df, test_df



In [ ]:
train, test = apply_arithmetic_operations(train, test, kek)

In [ ]:
final_features=[f for f in train.columns if f not in ['Target']]
final_features=[*set(final_features)]

sc=StandardScaler()

train_scaled=train.copy()
test_scaled=test.copy()
train_scaled[final_features]=sc.fit_transform(train[final_features])
test_scaled[final_features]=sc.transform(test[final_features])

In [ ]:
def post_processor(train, test):
    '''
    After Scaling, some of the features may be the same and can be eliminated
    '''
    cols=[f for f in train.columns if "Target" not in f and "OHE" not in f]
    train_cop=train.copy()
    test_cop=test.copy()
    drop_cols=[]
    for i, feature in enumerate(cols):
        for j in range(i+1, len(cols)):
            if sum(abs(train_cop[feature]-train_cop[cols[j]]))==0:
                if cols[j] not in drop_cols:
                    drop_cols.append(cols[j])
    print(drop_cols)
    train_cop.drop(columns=drop_cols,inplace=True)
    test_cop.drop(columns=drop_cols,inplace=True)
    
    return train_cop, test_cop

                    
train_cop, test_cop= post_processor(train_scaled, test_scaled)        

train_cop.to_csv('train_processed.csv',index=False)
test_cop.to_csv('test_processed.csv',index=False)

In [ ]:
X_train_full = train_cop.drop(columns=['Target'])
y_train_full = train['Target'].astype(int)


X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)


In [ ]:
classes = np.unique(y_train)  
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
y_train_numeric = np.array([class_to_index[cls] for cls in y_train])

class_counts = np.bincount(y_train_numeric)

total_samples = len(y_train_numeric)

class_weights = total_samples / (len(classes) * class_counts)

class_weights_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

print("Class counts:", class_counts)
print("Total samples:", total_samples)
print("Class weights:", class_weights)
print("Class weights dictionary:", class_weights_dict)

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from mlxtend.classifier import StackingClassifier
from sklearn.datasets import make_classification


xgb_classifier = XGBClassifier(n_estimators=300, random_state=42)
gbdt_classifier = GradientBoostingClassifier(n_estimators=300, random_state=42)
svc_classifier = SVC(C = 1.0, kernel='rbf', gamma='scale',probability=True)
rf_classifier = RandomForestClassifier(n_estimators=400, random_state=42)
#mlp_classifier = MLPClassifier(idden_layer_sizes=(128,64,32,16), activation='relu',solver='adam')
ada_classifier = AdaBoostClassifier(n_estimators=300, random_state=42)

# Create a StackingClassifier
stacked_classifier = StackingClassifier(
    classifiers=[xgb_classifier, gbdt_classifier, svc_classifier, rf_classifier,ada_classifier],
    meta_classifier=MLPClassifier(hidden_layer_sizes=(300, 150), max_iter=1000, random_state=165),
    use_probas=True,
    average_probas=False
)


stacked_classifier.fit(X_train, y_train)
y_pred = stacked_classifier.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy
print("Stacking Classifier Accuracy:", accuracy*100,"%")

In [ ]:
kekw = pd.read_csv('/kaggle/input/trainandtest/test.csv')
y_test_pred = stacked_classifier.predict(test_cop)

# Assuming test_cop contains 'ID' and other features
ids = kekw['id']

# Map the predicted encoded values to class labels
encoded_to_label = {0: 'Dropout', 1: 'Graduate', 2: 'Enrolled'}
y_test_pred_labels = [encoded_to_label[pred] for pred in y_test_pred]

# Create a DataFrame with 'ID' and 'Target' columns
submission = pd.DataFrame({'id': ids, 'Target': y_test_pred_labels})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print('Submission file created successfully.')